In [ ]:
llm_model = 'gpt-3.5-turbo-1106'

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

csv_file_path = 'better-members.csv'
data = CSVLoader(
    file_path=csv_file_path,
    encoding='utf-8',
    metadata_columns=['title', 'occupations', 'industries']
).load()

print(len(data))
data[:3]

210


[Document(page_content="name: 黒岩秀行\ncompany: ジムショDan代表\nprofile: Graduated from Keio University's Faculty of Economics in 1987, and joined Dentsu Co., Ltd. in the same year. From 1987 to 1997, worked as a copywriter and CM planner in the Creative Department, handling clients such as Ajinomoto, Coca-Cola Japan, Fumakilla, and Adelans. Received numerous awards including the Dentsu Award. From 1997 to 2020, held various positions in the Sales Department (currently Business Produce Department), working with clients such as Shiseido, Nippon Life, HIS, Yoshimoto Kogyo, and various publishing companies including Bungeishunju. Became the Sales Department Manager in 2005, focusing on planning and implementing large-scale campaigns. Currently, engaged as a member of the subsidiary of Dentsu, New Horizon Collective, under a business outsourcing contract. Graduated from the 43rd class of the 'Advertising Conference' editorial writer training course, receiving an award for outstanding graduation wo

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents=data, embedding=embedding
)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_model, temperature=0)

In [ ]:
import langchain
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, ConversationalRetrievalChain
from IPython.display import display, Pretty

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key='question',
    output_key='answer',
    return_messages=True
)
retriever = vectordb.as_retriever(
    search_kwargs={'k':5}
)

def chat(question, reset_memory=False, debug=False, verbose=False):
    langchain.debug = False
    if reset_memory: memory.clear()

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        verbose=verbose,
        return_source_documents=True
    )

    if debug:  # debug overcome verbose
        langchain.debug = True
        result = qa_chain({"question": question})
        langchain.debug = False
    else:
        result = qa_chain({"question": question})
        display(Pretty(result['answer']))

    return result

## Sensei's Questions

### Q0. Accounting

In [ ]:
result0 = chat("Please introduce me a specialist of accounting.", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: 江上徹
company: Leapfrog合同会社　代表
profile: Armed with the combination of 'accounting x English', I am capable of providing a wide range of services including consulting, internal and external audits, and corporate training. [Accounting] With approximately 10 years of experience in a certified public accounting firm and another 10 years in corporate accounting, I possess extensive practical experience in accounting, as well as profound insights in auditing, internal controls, etc. (Certified Public Accountant registration number: 41477). [English] In addition to a total of about 5 years of overseas assignments, during my tenure in the company, I regularly conducted video conferences as the contact po

I recommend 江上徹, who is a Certified Public Accountant with extensive experience in both accounting and English proficiency. He has approximately 10 years of experience in a certified public accounting firm and another 10 years in corporate accounting, making him a seasoned professional in the field. Additionally, he has experience in consulting, internal and external audits, and corporate training, making him a well-rounded specialist in accounting.

### Q1. Marketer

In [ ]:
result1 = chat("Can you please introduce me to a good marketer?", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: 宮本史郎
company: 
profile: I want to contribute to the development of marketing that does not rely on so-called 'selling' activities such as campaigns, promotions, discounts, and advertisements.

name: 田邉守
company: 田邉守
profile: Through over 30 years of practical experience as a business producer at Dentsu, I have acquired skills in marketing communication operations as well as an international business sense through a total of over 10 years of overseas assignments. Throughout my career as a business person, I have excelled in 1) effectively utilizing resources available in the 'field' to solve challenges, 2) creating 'new encounters and matching' for stakeholders, and 3) generating 'positive chemi

Based on the profiles provided, I would recommend introducing you to 二村暢朗, the marketing director at 株式会社ハビタットラボ. He has extensive experience in marketing and has worked with a wide range of clients, from major to emerging companies, across various industries. Additionally, he has expertise in consulting to address management issues and has been serving as a special professor at the Osaka Graduate School of Business. His experience and knowledge could be valuable in addressing your marketing needs.

### Q2. Creative specialists

In [ ]:
result2 = chat("Please tell us about creative specialists and their backgrounds", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: 市原弘一
company: dansima
profile: During my time at Dentsu, I consistently handled large-scale clients, contemplating what kind of thinking, imagery, and words are necessary to grow their brands. From the fundamental concept to the concrete expression, I was responsible for the entire process of brand development. I am eager to collaborate with hardworking entrepreneurs and decisive business leaders, regardless of company size. Let's savor the joys and pains of creation together. My official title is Creative Director, and depending on the project, I also work as a copywriter.

name: 京井良彦
company: 株式会社１／１　代表取締役
profile: Born in 1969. Worked at Dentsu for 20 years until 2020, then became independen

Here are some creative specialists and their backgrounds:

1. **市原弘一 (Dansima)**: Worked at Dentsu, handling large-scale clients and responsible for the entire process of brand development. Official title is Creative Director and also works as a copywriter.

2. **京井良彦 (株式会社１／１)**: Worked at Dentsu for 20 years, specializing in utilizing creativity beyond the advertising field, handling brand and marketing strategies, innovation business developments, and management strategies. Also a lecturer and author of books related to planning and presentation.

3. **宮下裕介 (Action Creative)**: Started as a copywriter at Dentsu, then became a creative director and photographer. Established Action Creative and excels in designing creatives that drive product movement, from top-funnel platforms like TV commercials to middle and bottom-funnel strategies.

4. **栗原敏雄 (KURIO creative partners Inc)**: Representative Director of KURIO creative partners Inc. While working at Dentsu, was responsible for devis

### Q3. Automobile industry

In [ ]:
result3 = chat("Are there any talented people in the automobile industry?", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: スズキユージ
company: 株式会社WHITE STUDIO
profile: Owns American cars for a quarter of a century. Currently struggling as a father of two. His wife is a photographer specializing in chocolate. They have a small photography studio. He works as the Creative Director for BMW/MINI and also for the minivan division.

name: 塩田京子
company: High touch
profile: I have extensive experience in talent development at Dentsu. I would like to help discover and deliver various hidden charms of people, things, and regions from various perspectives. I am also involved in challenging upcycling by connecting items that cannot be sold as regular products due to minor scratches with intellectual disability welfare workshops.


Yes, there are several talented individuals in the automobile industry. For example, 江坂俊弥 specializes in brand, sales, and public relations strategies for automotive brands, while 福井常晶 is a strategic planner and marketing consultant with extensive experience in planning and executing campaigns and marketing strategies primarily for automobiles. These individuals have a wealth of knowledge and experience in the automobile industry.

### Q4. Wide experience

In [ ]:
result4 = chat("Do you have people with a wide range of experience from human resources to sales?", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: 村上容
company: 
profile: I have been dedicated to sales. I have experience working in regional areas and have been involved in various industries such as automobiles, travel, daily necessities, and fashion, in various forms of work including media, events, and development. I was also a member of the Junior Chamber International (Osaka Junior Chamber, Japan Junior Chamber), which became a catalyst for considering becoming a freelancer. I also want to leverage the knowledge I gained at an advertising agency with a perspective similar to that of a small and medium-sized enterprise owner.

name: 田中信敬
company: 株式会社９Fire
profile: Engaged in solving customer issues as a business producer in BtoB sales a

Yes, we have several individuals with diverse experience in both human resources and sales. 村上容 has experience in sales across various industries, while 村松正基 transitioned from sales to HR and has been leading HR and labor affairs departments. Additionally, 孫麗 has extensive global HR management experience and has been involved in launching new businesses.

### Q5. Poster project

In [ ]:
result5 = chat("I have a project to make a poster for Sendai City, is there anyone who can help?", reset_memory=True, verbose=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
name: 関本みよ
company: 株式会社はな
profile: I handle almost anything other than commercials. I am looking forward to working on event projects. Many people consult me because they don't know who to talk to. I am a knitting enthusiast and I am researching the kitchen. I often work on projects related to food, regional areas, and cosmetics. I tend to plan reading materials.

name: 廣瀬誠
company: 
profile: I work in the role of connecting 'people, things, and events in the region' that are not concentrated in urban areas with 'someone, somewhere, or something somewhere.' By the way, my favorite things are Japanese swordsmanship (Shinkage-ryu martial arts) and Japanese traditional performing arts (Noh theater). I 

You may want to reach out to 関本みよ, who has experience in event projects and graphic design. She may be able to help you with creating a poster for Sendai City.

## Evaluation part

See [here](https://docs.ragas.io/en/latest/concepts/metrics/index.html) for explanation for each metric.

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_relevancy
from ragas.langchain import RagasEvaluatorChain

eval_chains = {
    m.name: RagasEvaluatorChain(metric=m)
    for m in [faithfulness, answer_relevancy, context_precision, context_relevancy]
}

def eval(result):
    result_dict = {}
    result['result'] = result['answer']
    result['query'] = result['question']
    for name, eval_chain in eval_chains.items():
        score_name = f"{name}_score"
        score = eval_chain(result)[score_name]
        result_dict[score_name] = score
        print(f"{score_name}: {score}")
    return result_dict

In [ ]:
results = [result0, result1, result2, result3, result4, result5]
results_dict = [{'question':res['question'], 'answer':res['answer']} for res in results]

def show_eval(question_id):
    result = results[question_id]
    print(f"Q{question_id+1}/6")
    print(result.get('answer'))
    print('---')
    result_dict = eval(result)
    results_dict[question_id].update(result_dict)

In [ ]:
show_eval(0)

Q1/6
I recommend 江上徹, who is a Certified Public Accountant with extensive experience in both accounting and English proficiency. He has approximately 10 years of experience in a certified public accounting firm and another 10 years in corporate accounting, making him a seasoned professional in the field. Additionally, he has experience in consulting, internal and external audits, and corporate training, making him a well-rounded specialist in accounting.
---
faithfulness_score: 1.0
answer_relevancy_score: 0.8582594149160423
context_precision_score: 0.9999999999666667
context_relevancy_score: 0.0


In [ ]:
show_eval(1)

Q2/6
Based on the profiles provided, I would recommend introducing you to 二村暢朗, the marketing director at 株式会社ハビタットラボ. He has extensive experience in marketing and has worked with a wide range of clients, from major to emerging companies, across various industries. Additionally, he has expertise in consulting to address management issues and has been serving as a special professor at the Osaka Graduate School of Business. His experience and knowledge could be valuable in addressing your marketing needs.
---
faithfulness_score: 1.0
answer_relevancy_score: 0.8722958435611594
context_precision_score: 0.249999999975
context_relevancy_score: 0.7741935483870968


In [ ]:
show_eval(2)

Q3/6
Here are some creative specialists and their backgrounds:

1. **市原弘一 (Dansima)**: Worked at Dentsu, handling large-scale clients and responsible for the entire process of brand development. Official title is Creative Director and also works as a copywriter.

2. **京井良彦 (株式会社１／１)**: Worked at Dentsu for 20 years, specializing in utilizing creativity beyond the advertising field, handling brand and marketing strategies, innovation business developments, and management strategies. Also a lecturer and author of books related to planning and presentation.

3. **宮下裕介 (Action Creative)**: Started as a copywriter at Dentsu, then became a creative director and photographer. Established Action Creative and excels in designing creatives that drive product movement, from top-funnel platforms like TV commercials to middle and bottom-funnel strategies.

4. **栗原敏雄 (KURIO creative partners Inc)**: Representative Director of KURIO creative partners Inc. While working at Dentsu, was responsible for 

In [ ]:
show_eval(3)

Q4/6
Yes, there are several talented individuals in the automobile industry. For example, 江坂俊弥 specializes in brand, sales, and public relations strategies for automotive brands, while 福井常晶 is a strategic planner and marketing consultant with extensive experience in planning and executing campaigns and marketing strategies primarily for automobiles. These individuals have a wealth of knowledge and experience in the automobile industry.
---
faithfulness_score: 1.0
answer_relevancy_score: 0.9895215446605471
context_precision_score: 0.0
context_relevancy_score: 0.03333333333333333


In [ ]:
show_eval(4)

Q5/6
Yes, we have several individuals with diverse experience in both human resources and sales. 村上容 has experience in sales across various industries, while 村松正基 transitioned from sales to HR and has been leading HR and labor affairs departments. Additionally, 孫麗 has extensive global HR management experience and has been involved in launching new businesses.
---
faithfulness_score: 1.0
answer_relevancy_score: 0.9705607092370858
context_precision_score: 0.4499999999775
context_relevancy_score: 0.0


In [ ]:
show_eval(5)

Q6/6
You may want to reach out to 関本みよ, who has experience in event projects and graphic design. She may be able to help you with creating a poster for Sendai City.
---
faithfulness_score: 0.3333333333333333
answer_relevancy_score: 0.963082671095895
context_precision_score: 0.0
context_relevancy_score: 0.0
